# Logistic Regression Code Along
This is a code along of the famous titanic dataset, its always nice to start off with this dataset because it is an example you will find across pretty much every data analysis language.

In [1]:
import findspark 
findspark.init('/home/gkouskosv/spark-2.4.5-bin-hadoop2.6/')

In [11]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('tit').getOrCreate()

In [6]:
data = spark.read.csv('titanic.csv', inferSchema=True, header=True)

In [7]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [8]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [9]:
my_cols = data.select(['Survived',
                       'Pclass',
                       'Sex',
                       'Age',
                       'SibSp',
                       'Parch',
                       'Fare',
                       'Embarked'])

In [10]:
my_final_data = my_cols.na.drop()

### Working with Categorical Columns

Let's break this down into multiple steps to make it all clear.

In [12]:
from pyspark.ml.feature import (VectorAssembler, StringIndexer,
                               VectorIndexer, OneHotEncoder)

In [13]:
gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [32]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkedIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkedIndex',outputCol='EmbarkedVec')

In [33]:
assembler = VectorAssembler(inputCols=['Pclass',
                                      'SexVec',
                                      'EmbarkedVec',
                                      'Age',
                                      'Parch',
                                      'Fare',
                                      'SibSp'],
                           outputCol='features')

In [20]:
from pyspark.ml.classification import LogisticRegression

## Pipelines 

Let's see an example of how to use pipelines (we'll get a lot more practice with these later!)

In [21]:
from pyspark.ml import Pipeline

In [26]:
log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')

In [34]:
pipeline = Pipeline(stages=[gender_indexer, gender_encoder,
                           embark_indexer, embark_encoder,
                           assembler, log_reg_titanic])

In [35]:
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7,0.3])

In [36]:
fit_model = pipeline.fit(train_titanic_data)

In [37]:
results = fit_model.transform(test_titanic_data)

In [38]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [40]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Survived')

In [41]:
results.select('survived','prediction').show()

+--------+----------+
|survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [45]:
AUC = my_eval.evaluate(results)

In [46]:
AUC

0.7732694892473118